# Finetune: Baseline (65 second)

In this notebook, we perform full finetuning on the PhysioNet dataset using a randomly initialized ResNet18. This will form the baseline to which we will compare the performance of the pre-trained ResNet18 on the Icentia11K dataset.

## Computational Requirements

We run this notebook in Google Colab Pro to utilize GPU resources. We perform finetuning using the **V100 GPU**.

## Data

We have prepared the PhysioNet data in a separate notebook ([Github](https://github.com/myles-i/DLH_TransferLearning/blob/master/jupyter_notebooks/finetuning_explore.ipynb)). That notebook generates _train_ and _test_ datasets.

In our case, the datasets are saved to our Google Drive at these paths:

1. `/content/drive/MyDrive/Project/data/physionet_finetune/physionet_train.pkl`
2. `/content/drive/MyDrive/Project/data/physionet_finetune/physionet_test.pkl`

On this data, the preprocessing is as follows:
1. Downsample the PhysioNet data to 250 hz
2. Pad recordings to ~65 second length
3. Normalize the recordings.


## Preparation

### Mount Google Drive

In [1]:
# You may also manually mount drive by clicking on folder icon in left sidebar
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Clone code repo

Assuming this notebook is run on Colab Pro, please clone our repo to the instance.

Sample commands to run in Colab Pro Terminal:

```bash
$ cd /root
# enter your username and github PAT
$ git clone https://github.com/myles-i/DLH_TransferLearning.git
$ cd DLH_TransferLearning
```

We will assume the code to run is on `master` branch.

### Install dependencies

We will install the dependencies from the `requirements.txt` file in the cloned repo.

In [2]:
REPO = '/root/DLH_TransferLearning/'
%cd $REPO

/root/DLH_TransferLearning


In [3]:
%%capture
! pip install -r requirements.txt

## Finetuning

Below we set up the paths to the:
1. `DATA_DIR`: location where the input files are
2. `JOB_DIR`: location where to save the output of the finetuning (model weights, history, etc.)

In [5]:
PROJECT_ROOT = '/content/drive/MyDrive/DLHProject'
DATA_DIR = PROJECT_ROOT + '/data'
JOB_DIR = PROJECT_ROOT + '/jobs'
! mkdir -p $JOB_DIR

Next, we set up the actual input and output paths relative to the `DATA_DIR` and `JOB_DIR`, respectively.

In [6]:
train = DATA_DIR + '/physionet_finetune/physionet_train.pkl'
test = DATA_DIR + '/physionet_finetune/physionet_test.pkl'
job_name = 'finetune_baseline_65sec'
job_dir = JOB_DIR + '/' + job_name

print(f"train: {train}")
print(f"test: {test}")
print(f"job_dir: {job_dir}")

train: /content/drive/MyDrive/DLHProject/data/physionet_finetune/physionet_train.pkl
test: /content/drive/MyDrive/DLHProject/data/physionet_finetune/physionet_test.pkl
job_dir: /content/drive/MyDrive/DLHProject/jobs/finetune_baseline_65sec


In [ ]:
job_dir = JOB_DIR + '/finetune_random_cnn_original_data_with_f1'
train = DATA_DIR + '/physionet_finetune/physionet_train.pkl'
test = DATA_DIR + '/physionet_finetune/physionet_test.pkl'

print(f"job_dir: {job_dir}")
print(f"train: {train}")
print(f"test: {test}")

job_dir: /content/drive/MyDrive/DLHProject/jobs/finetune_random_cnn_original_data_with_f1
train: /content/drive/MyDrive/DLHProject/data/physionet_finetune/physionet_train.pkl
test: /content/drive/MyDrive/DLHProject/data/physionet_finetune/physionet_test.pkl


The paper authors provide us this script that will perform the finetuning. We will run it below and expand on the meaning of the parameter values.

- `--val-size 0.0625`: This is the percentage of the train set size to set aside for the validation set.

  Note that the PhysioNet data was already split 80-20 train-test. The paper uses 5 percent of the full dataset for validation. We get this via $0.0625 * 0.8 = 0.05$
- `--val-metric "f1"`: Use macro F1 score to evaluate performance on validation set and to find the best model at each epoch.
- `--arch "resnet18"` means we are using ResNet18 CNN.
- `--batch-size 128` we found this value through trial and error that maximizes the utilization of the V100 GPU's 16 GB RAM.
- `--epochs 200` This is taken from the paper.
- `--seed 2024` this is the random seed used in splitting the train set into validation. This is for reproducibility.
- `--verbose` this prints out the finetuning progress epoch by epoch.


In [8]:
%%time
!python -m finetuning.trainer \
--job-dir $job_dir \
--train $train \
--test $test \
--val-size 0.0625 \
--val-metric "f1" \
--arch "resnet18" \
--batch-size 128 \
--epochs 200 \
--seed 2024 \
--verbose

2024-04-12 05:41:52.757898: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-12 05:41:52.757948: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-12 05:41:52.759349: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-12 05:41:52.767136: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-12 05:41:53.963730: W tensorflow/compiler/tf2

It took 30m 20s to run 72 epochs, as the paper authors have set up early stopping to end training if the loss on validation set does not decrease for more than 50 epochs.

